## Lorenz system (nonlinear ODE)


$$ \begin{aligned} \dot x &= \sigma(y-x)\\ \dot y &= x(\rho - z) - y \\ \dot z &= xy - \beta z, \end{aligned} $$


In [1]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.cm import rainbow
import numpy as np
from scipy.integrate import odeint
from scipy.io import loadmat
import pysindy as ps

In [2]:
# Generate training data
def lorenz(x, t):
    return [
        - 10 * x[0] - 10.9 * x[1],
        - 0.9 * x[1]  + 9.6 * x[0] * x[2],
        - 3.1 * x[0] * x[1] - 2.7 * x[2] - 7.1,
    ]

dt = 0.001
t_train = np.arange(0, 10, dt)
x0_train = [-8, 7, 27]
x_train = odeint(lorenz, x0_train, t_train)
x_dot_train_measured = np.array(
    [lorenz(x_train[i], 0) for i in range(t_train.size)]
)

In [3]:
# Candidate Functions 

def build_Theta(data, derivatives, derivatives_description, P, data_description = None):
    
    n,d = data.shape
    m, d2 = derivatives.shape
    if n != m: raise Exception('dimension error')
    if data_description is not None: 
        if len(data_description) != d: raise Exception('data descrption error')
    
    # Create a list of all polynomials in d variables up to degree P
    rhs_functions = {}
    f = lambda x, y : np.prod(np.power(list(x), list(y)))
    powers = []            
    for p in range(1,P+1):
            size = d + p - 1
            for indices in itertools.combinations(range(size), d-1):
                starts = [0] + [index+1 for index in indices]
                stops = indices + (size,)
                powers.append(tuple(map(operator.sub, stops, starts)))
    for power in powers: rhs_functions[power] = [lambda x, y = power: f(x,y), power]

    # First column of Theta is just ones.
    Theta = np.ones((n,1), dtype=np.complex64)
    descr = ['']
    
    # Add the derivaitves onto Theta
    for D in range(1,derivatives.shape[1]):
        Theta = np.hstack([Theta, derivatives[:,D].reshape(n,1)])
        descr.append(derivatives_description[D])
        
    # Add on derivatives times polynomials
    for D in range(derivatives.shape[1]):
        for k in rhs_functions.keys():
            func = rhs_functions[k][0]
            new_column = np.zeros((n,1), dtype=np.complex64)
            for i in range(n):
                new_column[i] = func(data[i,:])*derivatives[i,D]
            Theta = np.hstack([Theta, new_column])
            if data_description is None: descr.append(str(rhs_functions[k][1]) + derivatives_description[D])
            else:
                function_description = ''
                for j in range(d):
                    if rhs_functions[k][1][j] != 0:
                        if rhs_functions[k][1][j] == 1:
                            function_description = function_description + data_description[j]
                        else:
                            function_description = function_description + data_description[j] + '^' + str(rhs_functions[k][1][j])
                descr.append(function_description + derivatives_description[D])

    return Theta, descr

In [4]:
import itertools
import operator
# Form a huge matrix using up to quadratic polynomials in all variables.
X_data = x_train
X_ders = np.hstack([np.ones((10000,1))])
X_ders_descr = ['']
X, description = build_Theta(X_data, X_ders, X_ders_descr, 2, data_description = ['x','y','z'])
['1'] + description[1:]

['1', 'z', 'y', 'x', 'z^2', 'yz', 'y^2', 'xz', 'xy', 'x^2']

In [5]:
import scipy.io as sio
sio.savemat('Lorenz_v1.mat',{'Xt':x_dot_train_measured[:,0].reshape(10000,1),'Yt':x_dot_train_measured[:,1].reshape(10000,1),'Zt':x_dot_train_measured[:,2].reshape(10000,1),'R':X})